# Notebook 01: Data Collection

## Objective
Collect and prepare clean NBA data for player performance prediction.

## Data Pipeline
1. **data/raw/**: Original NBA API data (unmodified)
2. **data/processed/**: Cleaned and enriched data ready for feature engineering

## What This Notebook Does
1. **Collects raw data** from NBA API (game logs, shot charts, team stats)
2. **Enriches data** with opponent stats, team context, rest days
3. **Cleans data** (removes duplicates, orphans, standardizes naming)
4. **Saves to processed/** for use in feature engineering

## Output Files (data/processed/)
- `gamelogs_combined.parquet` - ~73,000 games with opponent/team context
- `shot_charts_all.parquet` - ~631,000 shots (deduplicated)
- Per-season game log files

## Key Features Added
- **Opponent stats**: DEF_RATING, OFF_RATING, PACE, W_PCT
- **Team stats**: Player's own team context  
- **Rest days**: DAYS_REST (-1 = first game for player, otherwise days since last game), IS_B2B
- **Clean data**: No duplicates, no orphaned records

In [1]:
import pandas as pd
import time
from pathlib import Path
from tqdm import tqdm
from nba_api.stats.endpoints import leaguegamelog, playergamelog, shotchartdetail

SEASONS = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
N_PLAYERS = 200
RATE_LIMIT = 0.6

# Create both raw and processed directories
raw_path = Path('../data/raw')
processed_path = Path('../data/processed')
raw_path.mkdir(parents=True, exist_ok=True)
processed_path.mkdir(parents=True, exist_ok=True)

print("Setup complete - imported NBA API and configured paths")
print(f"   data/raw/       - for unmodified API data")
print(f"   data/processed/ - for cleaned and enriched data")

Setup complete - imported NBA API and configured paths
   data/raw/       - for unmodified API data
   data/processed/ - for cleaned and enriched data


## Part 1: Collect Game Logs

In [2]:
# Step 1: Identify top 200 players from each season
print("Step 1: Identifying top 200 players per season based on total minutes played...\n")
top_players_by_season = {}

for season in SEASONS:
    print(f"{'='*60}\n{season}\n{'='*60}")
    
    # Get league game log for this season
    league_log = leaguegamelog.LeagueGameLog(
        season=season,
        season_type_all_star='Regular Season',
        player_or_team_abbreviation='P'
    )
    df_league = league_log.get_data_frames()[0]
    time.sleep(RATE_LIMIT)
    
    # Get top 200 players by total minutes
    top_players = (
        df_league.groupby(['PLAYER_ID', 'PLAYER_NAME'])['MIN']
        .sum()
        .reset_index()
        .sort_values('MIN', ascending=False)
        .head(N_PLAYERS)
    )
    
    top_players_by_season[season] = top_players
    print(f"Top player: {top_players.iloc[0]['PLAYER_NAME']} ({top_players.iloc[0]['MIN']:.0f} min)")
    print(f"Identified {len(top_players)} players for this season")

# Step 2: Get ALL unique players across all seasons
print(f"\n{'='*60}")
print("Step 2: Combining unique players across all seasons...")
print(f"{'='*60}\n")

all_top_players = pd.concat(top_players_by_season.values(), ignore_index=True)
unique_players = all_top_players.drop_duplicates(subset='PLAYER_ID')[['PLAYER_ID', 'PLAYER_NAME']]
print(f"Found {len(unique_players)} unique players across all {len(SEASONS)} seasons")

# Step 3: Collect game logs for ALL seasons for each unique player
print(f"\n{'='*60}")
print("Step 3: Collecting complete game logs for all identified players...")
print(f"This will take approximately 4-5 hours. Progress updates every 50 players.")
print(f"{'='*60}\n")

all_games = []
for idx, (_, player) in enumerate(tqdm(unique_players.iterrows(), total=len(unique_players), desc="Players")):
    player_id = player['PLAYER_ID']
    player_name = player['PLAYER_NAME']
    
    for season in SEASONS:
        try:
            gamelog = playergamelog.PlayerGameLog(
                player_id=player_id, 
                season=season, 
                season_type_all_star='Regular Season'
            )
            df_games = gamelog.get_data_frames()[0]
            
            if len(df_games) > 0:
                df_games['PLAYER_NAME'] = player_name
                all_games.append(df_games)
            
            time.sleep(RATE_LIMIT)
        except Exception as e:
            # Player didn't play in this season - skip silently
            time.sleep(1)
    
    # Progress update every 50 players
    if (idx + 1) % 50 == 0:
        total_games = sum(len(g) for g in all_games)
        print(f"\nProgress: {idx+1}/{len(unique_players)} players completed | {total_games:,} total games collected so far")

# Step 4: Combine and save
print(f"\n{'='*60}")
print("Step 4: Combining all data and saving to disk...")
print(f"{'='*60}\n")

df_all = pd.concat(all_games, ignore_index=True)

# Save combined file
df_all.to_parquet(raw_path / "gamelogs_combined.parquet", index=False)

# Also save per-season files for reference
print("Games per season breakdown:")
for season in SEASONS:
    season_data = df_all[df_all['SEASON_ID'].astype(str).str.contains(season.split('-')[0])]
    if len(season_data) > 0:
        season_data.to_parquet(raw_path / f"gamelogs_{season}.parquet", index=False)
        print(f"  {season}: {len(season_data):,} games")

print(f"\nFinal totals: {len(df_all):,} games from {df_all['Player_ID'].nunique()} unique players")
print(f"Data saved to data/raw/ directory")

Step 1: Identifying top 200 players per season based on total minutes played...

2019-20
Top player: CJ McCollum (2560 min)
Identified 200 players for this season
2020-21
Top player: Julius Randle (2666 min)
Identified 200 players for this season
2021-22
Top player: Mikal Bridges (2854 min)
Identified 200 players for this season
2022-23
Top player: Mikal Bridges (2965 min)
Identified 200 players for this season
2023-24
Top player: DeMar DeRozan (2995 min)
Identified 200 players for this season

Step 2: Combining unique players across all seasons...

Found 369 unique players across all 5 seasons

Step 3: Collecting complete game logs for all identified players...
This will take approximately 4-5 hours. Progress updates every 50 players.



Players:  14%|█▎        | 50/369 [04:07<26:30,  4.99s/it]


Progress: 50/369 players completed | 15,375 total games collected so far


Players:  27%|██▋       | 100/369 [08:32<25:13,  5.63s/it]


Progress: 100/369 players completed | 28,972 total games collected so far


...


Progress: 300/369 players completed | 66,869 total games collected so far


Players:  95%|█████████▍| 350/369 [3:02:39<01:41,  5.36s/it]   


Progress: 350/369 players completed | 72,299 total games collected so far


Players: 100%|██████████| 369/369 [3:39:24<00:00, 35.68s/it] 



Step 4: Combining all data and saving to disk...

Games per season breakdown:
  2019-20: 12,770 games
  2020-21: 14,026 games
  2021-22: 16,097 games
  2022-23: 15,120 games
  2023-24: 14,496 games

Final totals: 72,509 games from 289 unique players
Data saved to data/raw/ directory


## Part 2: Collect Shot Charts

In [3]:
players = df_all[['Player_ID', 'PLAYER_NAME']].drop_duplicates()
print(f"Collecting shot charts for {len(players)} players...\n")

all_shots = []
errors = []  # Track errors for debugging

for idx, (_, row) in enumerate(tqdm(players.iterrows(), total=len(players))):
    for season in SEASONS:
        try:
            shot_chart = shotchartdetail.ShotChartDetail(
                team_id=0,
                player_id=row['Player_ID'],
                season_nullable=season,
                season_type_all_star='Regular Season',
                context_measure_simple='FGA'
            )
            df_shots_temp = shot_chart.get_data_frames()[0]
            
            if len(df_shots_temp) > 0:
                # Add season identifier (API doesn't provide this)
                df_shots_temp['Season'] = season
                all_shots.append(df_shots_temp)
                
            time.sleep(RATE_LIMIT)
            
        except Exception as e:
            # Log error but continue (player may not have played this season)
            errors.append({
                'player_id': row['Player_ID'],
                'player_name': row['PLAYER_NAME'],
                'season': season,
                'error': str(e)
            })
            time.sleep(1)
    
    if (idx + 1) % 50 == 0:
        print(f"Progress: {idx+1}/{len(players)} | Shots: {sum(len(s) for s in all_shots):,}")

df_shots = pd.concat(all_shots, ignore_index=True)
df_shots.to_parquet(raw_path / "shot_charts_all.parquet", index=False)

print(f"\nCollected {len(df_shots):,} shots successfully")
if len(errors) > 0:
    print(f"WARNING: {len(errors)} API errors occurred (expected for players without shots in some seasons)")
    print(f"First 3 errors: {errors[:3]}")

 17%|█▋        | 50/289 [29:28<49:24, 12.40s/it]   

Progress: 50/289 | Shots: 170,885


 35%|███▍      | 100/289 [40:10<50:46, 16.12s/it] 

Progress: 100/289 | Shots: 321,785


 52%|█████▏    | 150/289 [1:24:09<33:04, 14.28s/it]    

...

Progress: 200/289 | Shots: 453,461


 87%|████████▋ | 250/289 [2:57:46<05:37,  8.65s/it]   

Progress: 250/289 | Shots: 546,489


100%|██████████| 289/289 [3:01:22<00:00, 37.65s/it]



Collected 597,106 shots successfully
First 3 errors: [{'player_id': 203468, 'player_name': 'CJ McCollum', 'season': '2019-20', 'error': 'HTTPSConnectionPool(host=\'stats.nba.com\', port=443): Max retries exceeded with url: /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=203468&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=2019-20&SeasonSegment=&SeasonType=Regular+Season&StartPeriod=&StartRange=&TeamID=0&VsConference=&VsDivision= (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x116a45810>: Failed to resolve \'stats.nba.com\' ([Errno 8] nodename nor servname provided, or not known)"))'}, {'player_id': 203468, 'player_name': 'CJ McCollum', 'season': '2020-21', 'error': 'HTTPSConnectionPool(host=\'stats.nba.com\', port=443): Max retries exceeded with url: /sta

## Part 3: Collect Opponent/Team Stats

Opponent strength is a critical contextual feature for prediction. We'll collect:
- **Defensive Rating (DEFRTG)**: Points allowed per 100 possessions
- **Offensive Rating (OFFRTG)**: Points scored per 100 possessions  
- **Pace**: Possessions per 48 minutes
- **Win/Loss record**: Team strength indicator

In [4]:
from nba_api.stats.endpoints import leaguedashteamstats

print("Collecting team stats for all teams across all seasons...\n")

all_team_stats = []
for season in SEASONS:
    print(f"{'='*60}\n{season}\n{'='*60}")
    
    try:
        # Get team stats for this season
        team_stats = leaguedashteamstats.LeagueDashTeamStats(
            season=season,
            season_type_all_star='Regular Season',
            measure_type_detailed_defense='Advanced',  # Get advanced stats (DRtg, ORtg, Pace)
            per_mode_detailed='PerGame'
        )
        df_team_stats = team_stats.get_data_frames()[0]
        
        # Add season identifier
        df_team_stats['SEASON'] = season
        all_team_stats.append(df_team_stats)
        
        print(f"Collected stats for {len(df_team_stats)} teams")
        time.sleep(RATE_LIMIT)
        
    except Exception as e:
        print(f"WARNING: Error collecting team stats for {season}: {e}")
        time.sleep(1)

# Combine all team stats
df_team_stats_all = pd.concat(all_team_stats, ignore_index=True)

# Save team stats
df_team_stats_all.to_parquet(raw_path / "team_stats_all.parquet", index=False)
print(f"\nTotal: {len(df_team_stats_all):,} team-season records")
print(f"Saved to data/raw/team_stats_all.parquet")


2019-20
Collected stats for 30 teams
2020-21
Collected stats for 30 teams
2021-22
Collected stats for 30 teams
2022-23
Collected stats for 30 teams
2023-24
Collected stats for 30 teams

Total: 150 team-season records
Saved to data/raw/team_stats_all.parquet


In [5]:
print("\n" + "="*60)
print("Merging Opponent Stats into Game Logs")
print("="*60)

# NBA team abbreviation to full name mapping
TEAM_ABBREV_TO_NAME = {
    'ATL': 'Atlanta Hawks',
    'BKN': 'Brooklyn Nets',
    'BOS': 'Boston Celtics',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'LA Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

# Extract opponent team abbreviation from MATCHUP column
def extract_opponent(matchup):
    """Extract opponent team abbreviation from MATCHUP string."""
    if ' @ ' in matchup:
        return matchup.split(' @ ')[1]
    elif ' vs. ' in matchup:
        return matchup.split(' vs. ')[1]
    else:
        return None

df_all['OPP_TEAM_ABBREV'] = df_all['MATCHUP'].apply(extract_opponent)
df_all['OPP_TEAM_NAME'] = df_all['OPP_TEAM_ABBREV'].map(TEAM_ABBREV_TO_NAME)

# Convert SEASON_ID to season format
def season_id_to_season(season_id):
    """Convert SEASON_ID (22019) to season format (2019-20)."""
    year = str(season_id)[1:]
    next_year = str(int(year) + 1)[-2:]
    return f"{year}-{next_year}"

df_all['SEASON'] = df_all['SEASON_ID'].apply(season_id_to_season)

print(f"\nMerging opponent stats into game logs...")
print(f"   Game logs before merge: {df_all.shape}")
print(f"   Team stats available: {df_team_stats_all.shape}")

# Select and rename opponent stats
df_opponent_stats = df_team_stats_all[['TEAM_NAME', 'SEASON', 'DEF_RATING', 'OFF_RATING', 
                                         'PACE', 'W', 'L', 'W_PCT']].copy()

print(f"\n   Sample team names in team stats: {df_opponent_stats['TEAM_NAME'].unique()[:5]}")
print(f"   Sample OPP_TEAM_NAME in game logs: {df_all['OPP_TEAM_NAME'].unique()[:5]}")
print(f"   Sample SEASON in team stats: {df_opponent_stats['SEASON'].unique()}")
print(f"   Sample SEASON in game logs: {df_all['SEASON'].unique()}")

# Merge on opponent team name + season
df_all = df_all.merge(
    df_opponent_stats,
    left_on=['OPP_TEAM_NAME', 'SEASON'],
    right_on=['TEAM_NAME', 'SEASON'],
    how='left',
    suffixes=('', '_OPP')
)

# Rename the merged columns to have OPP_ prefix
df_all = df_all.rename(columns={
    'DEF_RATING': 'OPP_DEF_RATING',
    'OFF_RATING': 'OPP_OFF_RATING',
    'PACE': 'OPP_PACE',
    'W': 'OPP_W',
    'L': 'OPP_L',
    'W_PCT': 'OPP_W_PCT'
})

# Drop redundant TEAM_NAME column
df_all = df_all.drop(columns=['TEAM_NAME'], errors='ignore')

print(f"\nMerge complete!")
print(f"   Game logs after merge: {df_all.shape}")

# Check merge success
if 'OPP_DEF_RATING' in df_all.columns:
    missing = df_all['OPP_DEF_RATING'].isnull().sum()
    if missing == 0:
        print(f"   SUCCESS: All {len(df_all):,} games have opponent stats")
    else:
        pct = missing / len(df_all) * 100
        print(f"   WARNING: {missing:,} games ({pct:.1f}%) missing opponent stats")
        
        # Debug: show unmapped teams
        unmapped = df_all[df_all['OPP_DEF_RATING'].isnull()]['OPP_TEAM_NAME'].value_counts()
        if len(unmapped) > 0:
            print(f"\n   Unmapped opponent teams:")
            for team, count in unmapped.items():
                print(f"      {team}: {count} games")
    
    # Show sample
    print(f"\nSample opponent stats:")
    sample = df_all[['MATCHUP', 'OPP_TEAM_ABBREV', 'OPP_DEF_RATING', 'OPP_OFF_RATING', 'OPP_PACE']].head(3)
    print(sample.to_string(index=False))
else:
    print(f"   ERROR: OPP_DEF_RATING column not created!")
    print(f"   Columns after merge: {df_all.columns.tolist()}")

print(f"\nOpponent-related columns added:")
opp_cols = [col for col in df_all.columns if 'OPP' in col.upper()]
print(f"   {opp_cols}")


Merging Opponent Stats into Game Logs

Merging opponent stats into game logs...
   Game logs before merge: (72509, 31)
   Team stats available: (150, 47)

   Sample team names in team stats: ['Atlanta Hawks' 'Boston Celtics' 'Brooklyn Nets' 'Charlotte Hornets'
 'Chicago Bulls']
   Sample OPP_TEAM_NAME in game logs: ['Brooklyn Nets' 'Dallas Mavericks' 'Philadelphia 76ers' 'LA Clippers'
 'Denver Nuggets']
   Sample SEASON in team stats: ['2019-20' '2020-21' '2021-22' '2022-23' '2023-24']
   Sample SEASON in game logs: ['2019-20' '2020-21' '2021-22' '2022-23' '2023-24']

Merge complete!
   Game logs after merge: (72509, 37)
   SUCCESS: All 72,509 games have opponent stats

Sample opponent stats:
    MATCHUP OPP_TEAM_ABBREV  OPP_DEF_RATING  OPP_OFF_RATING  OPP_PACE
  POR @ BKN             BKN           109.2           108.7    101.70
  POR @ DAL             DAL           111.2           115.9     99.89
POR vs. PHI             PHI           108.4           110.7     99.59

Opponent-related

In [6]:
print("\n" + "="*60)
print("Cleanup: Remove Duplicate Opponent Columns")
print("="*60)

# Remove duplicate columns with _x and _y suffixes
# These were created from running the merge cell multiple times
duplicate_suffixes = ['_x', '_y']
cols_to_drop = [col for col in df_all.columns if any(col.endswith(suffix) for suffix in duplicate_suffixes)]

if cols_to_drop:
    print(f"\nFound {len(cols_to_drop)} duplicate columns to remove:")
    for col in cols_to_drop:
        print(f"   - {col}")
    
    df_all = df_all.drop(columns=cols_to_drop)
    print(f"\nRemoved duplicate columns")
else:
    print(f"\nNo duplicate columns found")

# Also drop the old OPP_TEAM column if it exists (we use OPP_TEAM_ABBREV now)
if 'OPP_TEAM' in df_all.columns and 'OPP_TEAM_ABBREV' in df_all.columns:
    df_all = df_all.drop(columns=['OPP_TEAM'])
    print(f"Removed redundant OPP_TEAM column (keeping OPP_TEAM_ABBREV)")

print(f"\nFinal game logs shape: {df_all.shape}")
print(f"Final opponent columns:")
opp_cols = [col for col in df_all.columns if 'OPP' in col.upper()]
print(f"   {opp_cols}")

# Verify we have the key opponent stats
required_opp_cols = ['OPP_DEF_RATING', 'OPP_OFF_RATING', 'OPP_PACE', 'OPP_W_PCT']
missing_required = [col for col in required_opp_cols if col not in df_all.columns]

if missing_required:
    print(f"\nWARNING: Missing required columns: {missing_required}")
else:
    print(f"\nAll required opponent stat columns present!")
    print(f"\nOpponent stats summary:")
    print(df_all[required_opp_cols].describe())


Cleanup: Remove Duplicate Opponent Columns

No duplicate columns found

Final game logs shape: (72509, 37)
Final opponent columns:
   ['OPP_TEAM_ABBREV', 'OPP_TEAM_NAME', 'OPP_DEF_RATING', 'OPP_OFF_RATING', 'OPP_PACE', 'OPP_W', 'OPP_L', 'OPP_W_PCT']

All required opponent stat columns present!

Opponent stats summary:
       OPP_DEF_RATING  OPP_OFF_RATING      OPP_PACE     OPP_W_PCT
count    72509.000000    72509.000000  72509.000000  72509.000000
mean       112.407383      112.413351     99.609503      0.499810
std          3.117331        3.567888      1.933199      0.141794
min        102.500000      102.800000     95.640000      0.171000
25%        110.600000      110.100000     98.020000      0.415000
50%        112.300000      112.700000     99.410000      0.524000
75%        114.400000      114.600000    101.020000      0.610000
max        119.600000      122.200000    105.510000      0.780000


In [7]:
print("\n" + "="*60)
print("Adding Player's Own Team Stats")
print("="*60)

# Extract player's team from MATCHUP column
# MATCHUP format: "POR @ BKN" (POR is player's team) or "POR vs. LAC" (POR is player's team)
def extract_own_team(matchup):
    """Extract player's team abbreviation from MATCHUP string."""
    if ' @ ' in matchup:
        # Away game: "POR @ BKN" -> player's team is POR
        return matchup.split(' @ ')[0]
    elif ' vs. ' in matchup:
        # Home game: "POR vs. LAC" -> player's team is POR
        return matchup.split(' vs. ')[0]
    else:
        return None

df_all['TEAM_ABBREV'] = df_all['MATCHUP'].apply(extract_own_team)
df_all['TEAM_NAME'] = df_all['TEAM_ABBREV'].map(TEAM_ABBREV_TO_NAME)

print(f"\nMerging player's team stats...")
print(f"   Sample player teams: {df_all['TEAM_ABBREV'].unique()[:5]}")

# Select player's team stats
df_player_team_stats = df_team_stats_all[['TEAM_NAME', 'SEASON', 'DEF_RATING', 'OFF_RATING',
                                           'PACE', 'W', 'L', 'W_PCT']].copy()

# Merge on player's team name + season
df_all = df_all.merge(
    df_player_team_stats,
    left_on=['TEAM_NAME', 'SEASON'],
    right_on=['TEAM_NAME', 'SEASON'],
    how='left',
    suffixes=('', '_TEAM')
)

# Rename columns to have TEAM_ prefix
df_all = df_all.rename(columns={
    'DEF_RATING': 'TEAM_DEF_RATING',
    'OFF_RATING': 'TEAM_OFF_RATING',
    'PACE': 'TEAM_PACE',
    'W': 'TEAM_W',
    'L': 'TEAM_L',
    'W_PCT': 'TEAM_W_PCT'
})

print(f"\nMerged player's team stats!")
print(f"   Game logs after merge: {df_all.shape}")

# Verify merge
if 'TEAM_DEF_RATING' in df_all.columns:
    missing = df_all['TEAM_DEF_RATING'].isnull().sum()
    if missing == 0:
        print(f"   SUCCESS: All {len(df_all):,} games have team stats")
    else:
        pct = missing / len(df_all) * 100
        print(f"   WARNING: {missing:,} games ({pct:.1f}%) missing team stats")
        
    # Show sample
    print(f"\nSample team vs opponent comparison:")
    sample_cols = ['MATCHUP', 'TEAM_ABBREV', 'TEAM_DEF_RATING', 'OPP_DEF_RATING', 
                   'TEAM_OFF_RATING', 'OPP_OFF_RATING']
    print(df_all[sample_cols].head(3).to_string(index=False))
else:
    print(f"   ERROR: TEAM_DEF_RATING not created!")

print(f"\nTeam-related columns added:")
team_cols = [col for col in df_all.columns if 'TEAM_' in col]
print(f"   {team_cols}")


Adding Player's Own Team Stats

Merging player's team stats...
   Sample player teams: ['POR' 'NOP' 'PHX' 'SAC' 'HOU']

Merged player's team stats!
   Game logs after merge: (72509, 45)
   SUCCESS: All 72,509 games have team stats

Sample team vs opponent comparison:
    MATCHUP TEAM_ABBREV  TEAM_DEF_RATING  OPP_DEF_RATING  TEAM_OFF_RATING  OPP_OFF_RATING
  POR @ BKN         POR            114.3           109.2            113.2           108.7
  POR @ DAL         POR            114.3           111.2            113.2           115.9
POR vs. PHI         POR            114.3           108.4            113.2           110.7

Team-related columns added:
   ['OPP_TEAM_ABBREV', 'OPP_TEAM_NAME', 'TEAM_ABBREV', 'TEAM_NAME', 'TEAM_DEF_RATING', 'TEAM_OFF_RATING', 'TEAM_PACE', 'TEAM_W', 'TEAM_L', 'TEAM_W_PCT']


In [8]:
print("\n" + "="*60)
print("Adding Rest Days & Back-to-Back Indicators")
print("="*60)

# IMPORTANT: Convert GAME_DATE to datetime first (it's currently a string from API)
df_all['GAME_DATE'] = pd.to_datetime(df_all['GAME_DATE'])

# Sort by player and date to compute rest days
# NOTE: Using 'Player_ID' (mixed case) because standardization happens later in Cell 23
df_all = df_all.sort_values(['Player_ID', 'GAME_DATE']).reset_index(drop=True)

# Calculate days since last game for each player
df_all['DAYS_REST'] = df_all.groupby('Player_ID')['GAME_DATE'].diff().dt.days

# For first game of each player in our dataset, there's no previous game to compare
# Fill with -1 to indicate "first game in dataset for this player"
df_all['DAYS_REST'] = df_all['DAYS_REST'].fillna(-1).astype(int)

# Flag back-to-back games (games on consecutive days)
df_all['IS_B2B'] = (df_all['DAYS_REST'] == 1).astype(int)

print(f"\nRest days distribution:")
print(f"   Mean days rest: {df_all[df_all['DAYS_REST'] >= 0]['DAYS_REST'].mean():.1f}")
print(f"   Median days rest: {df_all[df_all['DAYS_REST'] >= 0]['DAYS_REST'].median():.0f}")
print(f"   First games (DAYS_REST = -1): {(df_all['DAYS_REST'] == -1).sum():,}")

print(f"\nBack-to-back statistics:")
b2b_count = df_all['IS_B2B'].sum()
b2b_pct = b2b_count / len(df_all) * 100
print(f"   Total B2B games: {b2b_count:,} ({b2b_pct:.1f}% of all games)")

print(f"\nRest days breakdown (excluding first games):")
rest_dist = df_all[df_all['DAYS_REST'] >= 0]['DAYS_REST'].value_counts().sort_index()
for days, count in rest_dist.head(7).items():
    pct = count / len(df_all) * 100
    print(f"   {days} day(s) rest: {count:,} games ({pct:.1f}%)")

print(f"\nRest days calculated successfully!")
print(f"   New columns: DAYS_REST, IS_B2B")
print(f"   Note: DAYS_REST = -1 indicates first game for that player in dataset")


Adding Rest Days & Back-to-Back Indicators

Rest days distribution:
   Mean days rest: 5.5
   Median days rest: 2
   First games (DAYS_REST = -1): 289

Back-to-back statistics:
   Total B2B games: 11,188 (15.4% of all games)

Rest days breakdown (excluding first games):
   1 day(s) rest: 11,188 games (15.4%)
   2 day(s) rest: 42,566 games (58.7%)
   3 day(s) rest: 10,303 games (14.2%)
   4 day(s) rest: 2,580 games (3.6%)
   5 day(s) rest: 988 games (1.4%)
   6 day(s) rest: 542 games (0.7%)
   7 day(s) rest: 433 games (0.6%)

Rest days calculated successfully!
   New columns: DAYS_REST, IS_B2B
   Note: DAYS_REST = -1 indicates first game for that player in dataset


In [9]:
print("\n" + "="*60)
print("Removing Redundant Columns")
print("="*60)

# Track columns before cleanup
cols_before = len(df_all.columns)

# Remove redundant columns
redundant_cols = []

# 1. Remove SEASON_ID (we have SEASON which is more readable)
if 'SEASON_ID' in df_all.columns and 'SEASON' in df_all.columns:
    df_all = df_all.drop(columns=['SEASON_ID'])
    redundant_cols.append('SEASON_ID')
    print(f"Removed SEASON_ID (keeping SEASON)")

# 2. Check for any remaining _x or _y suffixed columns from merges
duplicate_suffixes = ['_x', '_y', '_TEAM']
cols_to_check = [col for col in df_all.columns if any(col.endswith(suffix) for suffix in duplicate_suffixes)]

if cols_to_check:
    print(f"\nWARNING: Found {len(cols_to_check)} columns with merge suffixes:")
    for col in cols_to_check:
        print(f"   - {col}")

# 3. Drop VIDEO_AVAILABLE if it exists (not useful for prediction)
if 'VIDEO_AVAILABLE' in df_all.columns:
    df_all = df_all.drop(columns=['VIDEO_AVAILABLE'])
    redundant_cols.append('VIDEO_AVAILABLE')
    print(f"Removed VIDEO_AVAILABLE (not needed for modeling)")

cols_after = len(df_all.columns)
print(f"\nColumn count: {cols_before} -> {cols_after} ({cols_before - cols_after} removed)")

print(f"\nFinal column count by category:")
print(f"   Player info: {len([c for c in df_all.columns if 'PLAYER' in c.upper()])}")
print(f"   Game info: {len([c for c in df_all.columns if 'GAME' in c.upper()])}")
print(f"   Stats: {len([c for c in df_all.columns if any(x in c for x in ['PTS', 'REB', 'AST', 'FG', 'FT'])])}")
print(f"   Team context: {len([c for c in df_all.columns if 'TEAM_' in c])}")
print(f"   Opponent context: {len([c for c in df_all.columns if 'OPP_' in c])}")
print(f"   Rest/schedule: {len([c for c in df_all.columns if any(x in c for x in ['DAYS_REST', 'IS_B2B'])])}")

print(f"\nCleanup complete!")


Removing Redundant Columns
Removed SEASON_ID (keeping SEASON)
Removed VIDEO_AVAILABLE (not needed for modeling)

Column count: 47 -> 45 (2 removed)

Final column count by category:
   Player info: 2
   Game info: 2
   Stats: 14
   Team context: 10
   Opponent context: 8
   Rest/schedule: 2

Cleanup complete!


## Validation: Game Logs

Before proceeding, let's validate the game log data quality.

In [10]:
# 1. Check data shape and completeness
print("="*60)
print("GAME LOG VALIDATION")
print("="*60)

print("\n1. Dataset Shape:")
print(f"   Total games: {len(df_all):,}")
# NOTE: Using Player_ID (mixed case) - not renamed to PLAYER_ID until Cell 23
print(f"   Unique players: {df_all['Player_ID'].nunique()}")
print(f"   Columns: {df_all.shape[1]}")

print("\n2. Games per Season:")
# Use SEASON instead of SEASON_ID (which was dropped in Cell 12)
for season in sorted(df_all['SEASON'].unique()):
    count = (df_all['SEASON'] == season).sum()
    print(f"   {season}: {count:,} games")

print("\n3. Date Range:")
# GAME_DATE already converted to datetime in Cell 11
print(f"   Min: {df_all['GAME_DATE'].min()}")
print(f"   Max: {df_all['GAME_DATE'].max()}")

GAME LOG VALIDATION

1. Dataset Shape:
   Total games: 72,509
   Unique players: 289
   Columns: 45

2. Games per Season:
   2019-20: 12,770 games
   2020-21: 14,026 games
   2021-22: 16,097 games
   2022-23: 15,120 games
   2023-24: 14,496 games

3. Date Range:
   Min: 2019-10-22 00:00:00
   Max: 2024-04-14 00:00:00


In [11]:
# 2. Check for missing values in critical columns
print("\n4. Missing Values Check:")
# NOTE: Using Player_ID, Game_ID (mixed case) - not renamed until Cell 23
critical_cols = ['Player_ID', 'Game_ID', 'GAME_DATE', 'PTS', 'REB', 'AST', 'MIN', 'FGA', 'FG_PCT']
missing = df_all[critical_cols].isnull().sum()

if missing.sum() == 0:
    print("   No missing values in critical columns")
else:
    print("   WARNING: Missing values found:")
    for col, count in missing[missing > 0].items():
        print(f"      {col}: {count} ({count/len(df_all)*100:.2f}%)")

print("\n5. Duplicate Games Check:")
# Using Player_ID, Game_ID (mixed case)
duplicates = df_all.duplicated(subset=['Player_ID', 'Game_ID']).sum()
if duplicates == 0:
    print("   No duplicate player-game records found")
else:
    print(f"   WARNING: Found {duplicates} duplicate records")


4. Missing Values Check:
   No missing values in critical columns

5. Duplicate Games Check:
   No duplicate player-game records found


In [12]:
# 3. Validate data distributions
print("\n6. Target Variable Distributions:")
print(f"   PTS: {df_all['PTS'].mean():.1f} +/- {df_all['PTS'].std():.1f} (range: {df_all['PTS'].min()}-{df_all['PTS'].max()})")
print(f"   REB: {df_all['REB'].mean():.1f} +/- {df_all['REB'].std():.1f} (range: {df_all['REB'].min()}-{df_all['REB'].max()})")
print(f"   AST: {df_all['AST'].mean():.1f} +/- {df_all['AST'].std():.1f} (range: {df_all['AST'].min()}-{df_all['AST'].max()})")

print("\n7. Data Type Validation:")
# NOTE: Using Player_ID (mixed case) - not renamed until Cell 23
expected_types = {
    'Player_ID': 'int64',
    'PTS': 'int64',
    'REB': 'int64',
    'AST': 'int64',
    'FG_PCT': 'float64'
}

all_correct = True
for col, expected in expected_types.items():
    actual = str(df_all[col].dtype)
    if actual != expected:
        print(f"   WARNING: {col} - expected {expected}, got {actual}")
        all_correct = False

if all_correct:
    print("   All data types correct")

print("\n8. Required Columns Check:")
# NOTE: Using Player_ID, Game_ID (mixed case) and SEASON (SEASON_ID dropped in Cell 12)
required_cols = ['SEASON', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 
                 'MIN', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 'FTA', 'FT_PCT',
                 'REB', 'AST', 'PTS', 'TOV', 'PLAYER_NAME']
missing_cols = [col for col in required_cols if col not in df_all.columns]

if len(missing_cols) == 0:
    print(f"   All {len(required_cols)} required columns present")
else:
    print(f"   WARNING: Missing columns: {missing_cols}")


6. Target Variable Distributions:
   PTS: 13.2 +/- 9.2 (range: 0-73)
   REB: 4.8 +/- 3.6 (range: 0-31)
   AST: 3.0 +/- 2.8 (range: 0-24)

7. Data Type Validation:
   All data types correct

8. Required Columns Check:
   All 17 required columns present


In [13]:
print(f"\n11. Required Shot Columns Check:")
required_shot_cols = ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 
                      'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 
                      'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE',
                      'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE',
                      'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG',
                      'SHOT_MADE_FLAG', 'GAME_DATE', 'Season']

missing_shot_cols = [col for col in required_shot_cols if col not in df_shots.columns]

if len(missing_shot_cols) == 0:
    print(f"   All {len(required_shot_cols)} required shot columns present")
else:
    print(f"   WARNING: Missing columns: {missing_shot_cols}")

print(f"\n{'='*60}")
print("VALIDATION COMPLETE")
print(f"{'='*60}")
print(f"\nGame logs validated: {len(df_all):,} games")
print(f"Shot charts validated: {len(df_shots):,} shots")


11. Required Shot Columns Check:
   All 21 required shot columns present

VALIDATION COMPLETE

Game logs validated: 72,509 games
Shot charts validated: 597,106 shots


In [14]:
print(f"\n8. Shot Zone Distribution:")
zone_counts = df_shots['SHOT_ZONE_BASIC'].value_counts()
print(f"   Total zones: {len(zone_counts)}")
for zone, count in zone_counts.items():
    pct = count / len(df_shots) * 100
    print(f"   {zone}: {count:,} ({pct:.1f}%)")

print(f"\n9. Shot Made/Attempted Validation:")
total_attempted = df_shots['SHOT_ATTEMPTED_FLAG'].sum()
total_made = df_shots['SHOT_MADE_FLAG'].sum()
overall_fg_pct = (total_made / total_attempted * 100) if total_attempted > 0 else 0

print(f"   Total attempts: {total_attempted:,}")
print(f"   Total makes: {total_made:,}")
print(f"   Overall FG%: {overall_fg_pct:.1f}%")

# Sanity check: NBA league average is typically 45-47%
if 40 <= overall_fg_pct <= 50:
    print(f"   FG% in expected NBA range (40-50%)")
else:
    print(f"   WARNING: FG% outside typical NBA range - check data quality")

print(f"\n10. Shot Distance Distribution:")
print(f"   Min: {df_shots['SHOT_DISTANCE'].min()} ft")
print(f"   Max: {df_shots['SHOT_DISTANCE'].max()} ft")
print(f"   Mean: {df_shots['SHOT_DISTANCE'].mean():.1f} ft")
print(f"   Median: {df_shots['SHOT_DISTANCE'].median():.1f} ft")

# Sanity check: NBA court is 94 ft long, max shot distance ~90 ft
if df_shots['SHOT_DISTANCE'].max() <= 95:
    print(f"   All shot distances within court dimensions")
else:
    print(f"   WARNING: Some shots beyond court dimensions - check data quality")


8. Shot Zone Distribution:
   Total zones: 7
   Above the Break 3: 180,949 (30.3%)
   Restricted Area: 169,853 (28.4%)
   In The Paint (Non-RA): 109,020 (18.3%)
   Mid-Range: 82,185 (13.8%)
   Left Corner 3: 28,279 (4.7%)
   Right Corner 3: 25,697 (4.3%)
   Backcourt: 1,123 (0.2%)

9. Shot Made/Attempted Validation:
   Total attempts: 597,106
   Total makes: 278,752
   Overall FG%: 46.7%
   FG% in expected NBA range (40-50%)

10. Shot Distance Distribution:
   Min: 0 ft
   Max: 87 ft
   Mean: 13.9 ft
   Median: 14.0 ft
   All shot distances within court dimensions


In [15]:
print("\n5. Missing Values Check:")
shot_critical_cols = ['PLAYER_ID', 'GAME_ID', 'GAME_DATE', 'SHOT_ZONE_BASIC', 
                      'SHOT_DISTANCE', 'SHOT_MADE_FLAG', 'SHOT_ATTEMPTED_FLAG']
missing_shots = df_shots[shot_critical_cols].isnull().sum()

if missing_shots.sum() == 0:
    print("   No missing values in critical shot columns")
else:
    print("   WARNING: Missing values found:")
    for col, count in missing_shots[missing_shots > 0].items():
        print(f"      {col}: {count} ({count/len(df_shots)*100:.2f}%)")

print("\n6. Duplicate Shots Check:")
shot_duplicates = df_shots.duplicated(subset=['PLAYER_ID', 'GAME_ID', 'PERIOD', 
                                               'MINUTES_REMAINING', 'SECONDS_REMAINING']).sum()
if shot_duplicates == 0:
    print("   No duplicate shot records found")
else:
    print(f"   WARNING: Found {shot_duplicates} potential duplicate shots")
    pct = shot_duplicates / len(df_shots) * 100
    print(f"      ({pct:.3f}% of total - likely edge cases)")

print("\n7. Orphaned Shots Check:")
print("   Checking if all shots have corresponding game logs...")
# NOTE: df_all still has Player_ID, Game_ID (mixed case) at this point - not renamed until Cell 23
# df_shots has PLAYER_ID, GAME_ID (all caps) from NBA API
valid_player_game_pairs = set(zip(df_all['Player_ID'], df_all['Game_ID']))
shot_player_game_pairs = set(zip(df_shots['PLAYER_ID'], df_shots['GAME_ID']))
orphaned = len(shot_player_game_pairs - valid_player_game_pairs)

if orphaned == 0:
    print("   All shots have corresponding game logs")
else:
    print(f"   WARNING: {orphaned} player-game pairs in shots but not in game logs")
    pct = orphaned / len(shot_player_game_pairs) * 100
    print(f"      ({pct:.1f}% of player-game pairs)")
    print("      Note: Orphans are typically from players with < 1 minute played")


5. Missing Values Check:
   No missing values in critical shot columns

6. Duplicate Shots Check:
      (0.075% of total - likely edge cases)

7. Orphaned Shots Check:
   Checking if all shots have corresponding game logs...
      (0.8% of player-game pairs)
      Note: Orphans are typically from players with < 1 minute played


In [16]:
print("="*60)
print("SHOT CHART VALIDATION")
print("="*60)

print("\n1. Dataset Shape:")
print(f"   Total shots: {len(df_shots):,}")
# NOTE: Shots have PLAYER_ID (all caps) from NBA API
print(f"   Unique players: {df_shots['PLAYER_ID'].nunique()}")
print(f"   Unique games: {df_shots['GAME_ID'].nunique()}")
print(f"   Columns: {df_shots.shape[1]}")

print("\n2. Shots per Season:")
for season in SEASONS:
    count = (df_shots['Season'] == season).sum()
    print(f"   {season}: {count:,} shots")

print("\n3. Date Range:")
df_shots['GAME_DATE'] = pd.to_datetime(df_shots['GAME_DATE'])
print(f"   Min: {df_shots['GAME_DATE'].min()}")
print(f"   Max: {df_shots['GAME_DATE'].max()}")

print("\n4. Date Alignment Check:")
shots_date_range = (df_shots['GAME_DATE'].min(), df_shots['GAME_DATE'].max())
games_date_range = (df_all['GAME_DATE'].min(), df_all['GAME_DATE'].max())
if shots_date_range[0] >= games_date_range[0] and shots_date_range[1] <= games_date_range[1]:
    print("   Shot dates align with game log dates")
else:
    print(f"   WARNING: Date mismatch")
    print(f"   Shot dates: {shots_date_range}")
    print(f"   Game dates: {games_date_range}")

SHOT CHART VALIDATION

1. Dataset Shape:
   Total shots: 597,106
   Unique players: 229
   Unique games: 5829
   Columns: 25

2. Shots per Season:
   2019-20: 111,955 shots
   2020-21: 106,492 shots
   2021-22: 130,284 shots
   2022-23: 124,566 shots
   2023-24: 123,809 shots

3. Date Range:
   Min: 2019-10-22 00:00:00
   Max: 2024-04-14 00:00:00

4. Date Alignment Check:
   Shot dates align with game log dates


## Data Cleaning

Now that we've validated the raw data, let's clean it up for feature engineering:
1. **Fix duplicate columns** in shot charts (PLAYER_ID vs Player_ID)
2. **Standardize column naming** across both datasets
3. **Remove duplicate shot records** (574 found)
4. **Filter orphaned shots** without corresponding game logs (664 player-game pairs)

In [17]:
print("="*60)
print("STEP 1: Fix Duplicate Columns in Shot Charts")
print("="*60)

# Check current columns
print(f"\nCurrent shot chart columns ({len(df_shots.columns)} total):")
player_cols = [col for col in df_shots.columns if 'PLAYER' in col.upper()]
print(f"   Player-related columns: {player_cols}")

# The NBA API already provides PLAYER_ID and PLAYER_NAME
# We mistakenly added Player_ID and Player_Name duplicates in cell 5
# Let's remove the manually added ones and keep the API originals

if 'PLAYER_ID' in df_shots.columns and 'Player_ID' in df_shots.columns:
    # Verify they're actually duplicates
    id_mismatch = (df_shots['PLAYER_ID'] != df_shots['Player_ID']).sum()
    if id_mismatch == 0:
        print(f"\nVerified: PLAYER_ID and Player_ID are identical (0 mismatches)")
        df_shots = df_shots.drop(columns=['Player_ID'])
        print(f"   Dropped duplicate 'Player_ID' column")
    else:
        print(f"\nWARNING: Found {id_mismatch} mismatches between PLAYER_ID and Player_ID")

if 'PLAYER_NAME' in df_shots.columns and 'Player_Name' in df_shots.columns:
    name_mismatch = (df_shots['PLAYER_NAME'] != df_shots['Player_Name']).sum()
    if name_mismatch == 0:
        print(f"Verified: PLAYER_NAME and Player_Name are identical (0 mismatches)")
        df_shots = df_shots.drop(columns=['Player_Name'])
        print(f"   Dropped duplicate 'Player_Name' column")
    else:
        print(f"\nWARNING: Found {name_mismatch} mismatches between PLAYER_NAME and Player_Name")

print(f"\nAfter cleanup: {len(df_shots.columns)} columns remaining")
print(f"   Player columns: {[col for col in df_shots.columns if 'PLAYER' in col.upper()]}")

STEP 1: Fix Duplicate Columns in Shot Charts

Current shot chart columns (25 total):
   Player-related columns: ['PLAYER_ID', 'PLAYER_NAME']

After cleanup: 25 columns remaining
   Player columns: ['PLAYER_ID', 'PLAYER_NAME']


In [18]:
print("\n" + "="*60)
print("STEP 2: Standardize Column Naming")
print("="*60)

# Game logs have Player_ID and Game_ID (mixed case from NBA API)
# Shot charts have PLAYER_ID (all caps from NBA API)
# Let's standardize everything to UPPERCASE to match NBA API convention

print("\nBefore standardization:")
print(f"   Game logs: Player_ID, Game_ID")
print(f"   Shot charts: PLAYER_ID, GAME_ID")

# Rename game log columns to uppercase
rename_mapping = {
    'Player_ID': 'PLAYER_ID',
    'Game_ID': 'GAME_ID'
}

# Only rename if the mixed-case version exists
cols_to_rename = {k: v for k, v in rename_mapping.items() if k in df_all.columns}
if cols_to_rename:
    df_all = df_all.rename(columns=cols_to_rename)
    print(f"\nRenamed {len(cols_to_rename)} columns in game logs:")
    for old, new in cols_to_rename.items():
        print(f"   {old} -> {new}")
else:
    print(f"\nGame logs already use standard naming")

print(f"\nAfter standardization:")
print(f"   Game logs: {[col for col in df_all.columns if 'PLAYER' in col.upper() or 'GAME' in col.upper()][:5]}")
print(f"   Shot charts: {[col for col in df_shots.columns if 'PLAYER' in col.upper() or 'GAME' in col.upper()][:5]}")


STEP 2: Standardize Column Naming

Before standardization:
   Game logs: Player_ID, Game_ID
   Shot charts: PLAYER_ID, GAME_ID

Renamed 2 columns in game logs:
   Player_ID -> PLAYER_ID
   Game_ID -> GAME_ID

After standardization:
   Game logs: ['PLAYER_ID', 'GAME_ID', 'GAME_DATE', 'PLAYER_NAME']
   Shot charts: ['GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'GAME_DATE']


In [19]:
print("\n" + "="*60)
print("STEP 3: Remove Duplicate Shots")
print("="*60)

print(f"\nBefore deduplication: {len(df_shots):,} shots")

# Remove duplicates based on unique shot identifiers
# A shot is uniquely identified by: Player, Game, Period, and exact time
before_count = len(df_shots)
df_shots = df_shots.drop_duplicates(subset=['PLAYER_ID', 'GAME_ID', 'PERIOD', 
                                              'MINUTES_REMAINING', 'SECONDS_REMAINING'])
after_count = len(df_shots)
removed = before_count - after_count

if removed > 0:
    print(f"\nRemoved {removed:,} duplicate shot records")
    pct = removed / before_count * 100
    print(f"   ({pct:.3f}% of original data)")
else:
    print(f"\nNo duplicates found")

print(f"\nAfter deduplication: {len(df_shots):,} shots")


STEP 3: Remove Duplicate Shots

Before deduplication: 597,106 shots

Removed 445 duplicate shot records
   (0.075% of original data)

After deduplication: 596,661 shots


In [20]:
print("\n" + "="*60)
print("STEP 4: Filter Orphaned Shots")
print("="*60)

# Orphaned shots = shots without corresponding game logs
# These occur when players had shots but the game log API didn't return that game
# (typically players with <1 min played)

print(f"\nBefore filtering: {len(df_shots):,} shots")

# Create set of valid player-game pairs from game logs
valid_pairs = set(zip(df_all['PLAYER_ID'], df_all['GAME_ID']))

# Filter shots to only keep those with corresponding game logs
before_count = len(df_shots)
df_shots = df_shots[df_shots.apply(lambda x: (x['PLAYER_ID'], x['GAME_ID']) in valid_pairs, axis=1)]
after_count = len(df_shots)
removed = before_count - after_count

if removed > 0:
    print(f"\nRemoved {removed:,} orphaned shots")
    pct = removed / before_count * 100
    print(f"   ({pct:.3f}% of shots without corresponding game logs)")
else:
    print(f"\nNo orphaned shots found")

print(f"\nAfter filtering: {len(df_shots):,} shots")


STEP 4: Filter Orphaned Shots

Before filtering: 596,661 shots

Removed 5,194 orphaned shots
   (0.871% of shots without corresponding game logs)

After filtering: 591,467 shots


In [21]:
print("\n" + "="*60)
print("STEP 5: Save Cleaned & Enriched Data to Processed Directory")
print("="*60)

# Save cleaned datasets to data/processed/ (not raw/ - this data has been significantly modified)
print(f"\nSaving to data/processed/ directory...")
print(f"   (Original raw API data remains in data/raw/)")

# Save cleaned game logs
df_all.to_parquet(processed_path / "gamelogs_combined.parquet", index=False)
print(f"\nSaved cleaned game logs:")
print(f"   File: data/processed/gamelogs_combined.parquet")
print(f"   Rows: {len(df_all):,}")
print(f"   Columns: {df_all.shape[1]}")

# Save per-season files with cleaned data
# Use SEASON column (not SEASON_ID which was dropped in Cell 12)
print(f"\nPer-season breakdowns:")
for season in SEASONS:
    season_data = df_all[df_all['SEASON'] == season]
    if len(season_data) > 0:
        season_data.to_parquet(processed_path / f"gamelogs_{season}.parquet", index=False)
        print(f"   {season}: {len(season_data):,} games")

# Save cleaned shot charts
df_shots.to_parquet(processed_path / "shot_charts_all.parquet", index=False)
print(f"\nSaved cleaned shot charts:")
print(f"   File: data/processed/shot_charts_all.parquet")
print(f"   Rows: {len(df_shots):,}")
print(f"   Columns: {df_shots.shape[1]}")

# Final summary
print(f"\n{'='*60}")
print("DATA PROCESSING COMPLETE")
print("="*60)

print(f"\nFinal Clean Datasets:")
print(f"   Game logs: {len(df_all):,} games from {df_all['PLAYER_ID'].nunique()} players")
print(f"   Shot charts: {len(df_shots):,} shots from {df_shots['PLAYER_ID'].nunique()} players")
print(f"   Unique player-game pairs in shots: {len(set(zip(df_shots['PLAYER_ID'], df_shots['GAME_ID']))):,}")

print(f"\nWhat's been added to the data:")
print(f"   - Opponent stats (8 columns)")
print(f"   - Player's team stats (6 columns)")
print(f"   - Rest days & back-to-back indicators (2 columns)")
print(f"   - Removed duplicates and orphaned records")
print(f"   - Standardized column naming")

print(f"\nNext step: Feature engineering (rolling averages, lagged features, etc.)")


STEP 5: Save Cleaned & Enriched Data to Processed Directory

Saving to data/processed/ directory...
   (Original raw API data remains in data/raw/)

Saved cleaned game logs:
   File: data/processed/gamelogs_combined.parquet
   Rows: 72,509
   Columns: 45

Per-season breakdowns:
   2019-20: 12,770 games
   2020-21: 14,026 games
   2021-22: 16,097 games
   2022-23: 15,120 games
   2023-24: 14,496 games

Saved cleaned shot charts:
   File: data/processed/shot_charts_all.parquet
   Rows: 591,467
   Columns: 25

DATA PROCESSING COMPLETE

Final Clean Datasets:
   Game logs: 72,509 games from 289 players
   Shot charts: 591,467 shots from 229 players
   Unique player-game pairs in shots: 55,949

What's been added to the data:
   - Opponent stats (8 columns)
   - Player's team stats (6 columns)
   - Rest days & back-to-back indicators (2 columns)
   - Removed duplicates and orphaned records
   - Standardized column naming

Next step: Feature engineering (rolling averages, lagged features, etc

In [22]:
# Verify the processed data was saved correctly
df_verify = pd.read_parquet(processed_path / "gamelogs_combined.parquet")
print(f"Verified: Processed file has {df_verify.shape[1]} columns and {len(df_verify):,} rows")

print(f"\nOpponent context columns:")
opp_cols = [c for c in df_verify.columns if 'OPP_' in c]
for col in opp_cols:
    print(f"   - {col}")

print(f"\nTeam context columns:")
team_cols = [c for c in df_verify.columns if 'TEAM_' in c]
for col in team_cols:
    print(f"   - {col}")

print(f"\nRest/schedule columns:")
rest_cols = [c for c in df_verify.columns if any(x in c for x in ['DAYS_REST', 'IS_B2B'])]
for col in rest_cols:
    print(f"   - {col}")

# Show a chronologically sorted sample for clarity
print(f"\nSample data (sorted by player and date):")
sample_cols = ['PLAYER_NAME', 'GAME_DATE', 'MATCHUP', 'PTS', 'OPP_DEF_RATING', 'TEAM_DEF_RATING', 'DAYS_REST', 'IS_B2B']
sample_df = df_verify.sort_values(['PLAYER_ID', 'GAME_DATE']).head(5)
print(sample_df[sample_cols].to_string(index=False))

print(f"\nNote: DAYS_REST = -1 indicates first game for that player in our dataset")
print(f"\nData ready for feature engineering in next notebook!")

Verified: Processed file has 45 columns and 72,509 rows

Opponent context columns:
   - OPP_TEAM_ABBREV
   - OPP_TEAM_NAME
   - OPP_DEF_RATING
   - OPP_OFF_RATING
   - OPP_PACE
   - OPP_W
   - OPP_L
   - OPP_W_PCT

Team context columns:
   - OPP_TEAM_ABBREV
   - OPP_TEAM_NAME
   - TEAM_ABBREV
   - TEAM_NAME
   - TEAM_DEF_RATING
   - TEAM_OFF_RATING
   - TEAM_PACE
   - TEAM_W
   - TEAM_L
   - TEAM_W_PCT

Rest/schedule columns:
   - DAYS_REST
   - IS_B2B

Sample data (sorted by player and date):
 PLAYER_NAME  GAME_DATE     MATCHUP  PTS  OPP_DEF_RATING  TEAM_DEF_RATING  DAYS_REST  IS_B2B
LeBron James 2019-10-22   LAL @ LAC   18           106.9            106.1         -1       0
LeBron James 2019-10-25 LAL vs. UTA   32           109.3            106.1          3       0
LeBron James 2019-10-27 LAL vs. CHA   20           112.8            106.1          2       0
LeBron James 2019-10-29 LAL vs. MEM   23           109.7            106.1          2       0
LeBron James 2019-11-01   LAL @ DAL 

# Final Dataset Verification

In [23]:
print("="*60)
print("FINAL DATASET VERIFICATION")
print("="*60)

# Reload from processed directory to verify
df_test = pd.read_parquet(processed_path / "gamelogs_combined.parquet")

print(f"\nFinal Processed Dataset:")
print(f"   Location: data/processed/gamelogs_combined.parquet")
print(f"   Rows: {len(df_test):,}")
print(f"   Columns: {df_test.shape[1]}")

print(f"\nEnrichments successfully added:")
print(f"   Opponent stats: {len([c for c in df_test.columns if 'OPP_' in c])} columns")
print(f"   Team stats: {len([c for c in df_test.columns if 'TEAM_' in c])} columns")
print(f"   Schedule features: {len([c for c in df_test.columns if any(x in c for x in ['DAYS_REST', 'IS_B2B'])])} columns")

# Show chronologically sorted sample
print(f"\nSample data (first 3 games for first player):")
sample = df_test.sort_values(['PLAYER_ID', 'GAME_DATE']).head(3)
print(sample[['PLAYER_NAME', 'GAME_DATE', 'MATCHUP', 'PTS', 'DAYS_REST', 'OPP_DEF_RATING']].to_string(index=False))

print(f"\nData collection and processing complete!")
print(f"Ready for notebook 02: Feature Engineering")

FINAL DATASET VERIFICATION

Final Processed Dataset:
   Location: data/processed/gamelogs_combined.parquet
   Rows: 72,509
   Columns: 45

Enrichments successfully added:
   Opponent stats: 8 columns
   Team stats: 10 columns
   Schedule features: 2 columns

Sample data (first 3 games for first player):
 PLAYER_NAME  GAME_DATE     MATCHUP  PTS  DAYS_REST  OPP_DEF_RATING
LeBron James 2019-10-22   LAL @ LAC   18         -1           106.9
LeBron James 2019-10-25 LAL vs. UTA   32          3           109.3
LeBron James 2019-10-27 LAL vs. CHA   20          2           112.8

Data collection and processing complete!
Ready for notebook 02: Feature Engineering
